In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover, StringIndexer
from sklearn.model_selection import train_test_split

In [0]:
df = spark.read.options(header="True") \
.csv("s3://kadentranbigdata/Tweets.csv")
df.show(10)

+--------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+----------+-------------------+-------------+--------------------+-----------+--------------------+----------------+--------------------+
 tweet_id|airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence| airline|airline_sentiment_gold| name|negativereason_gold|retweet_count| text|tweet_coord| tweet_created| tweet_location| user_timezone|
+--------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+----------+-------------------+-------------+--------------------+-----------+--------------------+----------------+--------------------+
 570306133677760513| neutral| 1.0| null| null|Virgin America| null| cairdin| null| 0|@VirginAmerica Wh...| null|2015-02-24 11:35:...| null|Eastern Time (US ...|
 570301130888122368| positive| 0.3486| null| 0.0|Virgin America| null| jnardino| null| 0|@VirginAmerica pl...| null|2015-02-24 11:15:...| null|Pacific Time (US ...|
 570301083672813571| neutral| 0.6837| null| null|Virgin America| null|yvonnalynn| null| 0|@VirginAmerica I ...| null|2015-02-24 11:15:...| Lets Play|Central Time (US ...|
 570301031407624196| negative| 1.0| Bad Flight| 0.7033|Virgin America| null| jnardino| null| 0|"@VirginAmerica i...| null|2015-02-24 11:15:...| null|Pacific Time (US ...|
 570300817074462722| negative| 1.0| Can't Tell| 1.0|Virgin America| null| jnardino| null| 0|@VirginAmerica an...| null|2015-02-24 11:14:...| null|Pacific Time (US ...|
 570300767074181121| negative| 1.0| Can't Tell| 0.6842|Virgin America| null| jnardino| null| 0|@VirginAmerica se...| null| null| null| null|
it's really the o...| null| 2015-02-24 11:14:...| null| Pacific Time (US ...| null| null| null| null| null| null| null| null| null| null|
 570300616901320704| positive| 0.6745| null| 0.0|Virgin America| null|cjmcginnis| null| 0|@VirginAmerica ye...| null|2015-02-24 11:13:...|San Francisco CA|Pacific Time (US ...|
 570300248553349120| neutral| 0.634| null| null|Virgin America| null| pilot| null| 0|@VirginAmerica Re...| null|2015-02-24 11:12:...| Los Angeles|Pacific Time (US ...|
 570299953286942721| positive| 0.6559| null| null|Virgin America| null| dhepburn| null| 0|@virginamerica We...| null|2015-02-24 11:11:...| San Diego|Pacific Time (US ...|
+--------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+----------+-------------------+-------------+--------------------+-----------+--------------------+----------------+--------------------+
only showing top 10 rows

In [0]:
df.describe()

Out[81]: DataFrame[summary: string, tweet_id: string, airline_sentiment: string, airline_sentiment_confidence: string, negativereason: string, negativereason_confidence: string, airline: string, airline_sentiment_gold: string, name: string, negativereason_gold: string, retweet_count: string, text: string, tweet_coord: string, tweet_created: string, tweet_location: string, user_timezone: string]

In [0]:
#remove rows where the text field is null.
#df.na.drop("all", Seq("text")).show(false)
df = df.filter(df.text.isNotNull())
df.count()

Out[82]: 14632

In [0]:
#Split datase to train and test
train, test = df.randomSplit(weights=[0.8,0.2], seed =200)

print(f"No. of training examples: {train.count()}")
print(f"No. of testing examples: {test.count()}")

No. of training examples: 11718
No. of testing examples: 2914

In [0]:
#Configure ML pipeline with these stages: tokenizer, stop word remover, term hashing, label conversion
tokenizer = Tokenizer(inputCol="text", outputCol="words1")
stopwordsremover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="words2")
tf = HashingTF(inputCol=stopwordsremover.getOutputCol(), outputCol="features")
sentimentIndexer = StringIndexer(inputCol="airline_sentiment",outputCol="label")

lr = LogisticRegression(maxIter=20,labelCol='label')
#lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer,stopwordsremover,tf,sentimentIndexer, lr])

In [0]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
paramGrid = ParamGridBuilder() \
  .addGrid(tf.numFeatures,[10,100,1000]) \
  .addGrid(lr.regParam, [0.1, 0.01,0.001]) \
  .build()

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=2)

In [0]:
#train = train.withColumnRenamed("sentimentIndex","label")
# Fit the pipeline to training documents.
cvmodel = crossval.fit(train)
#train.show()}

/databricks/spark/python/pyspark/ml/util.py:838: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
# validate with test data
transformed = cvmodel.transform(test)
selected = transformed.select("airline_sentiment", "label", "text", "probability", "prediction")
selected.show()

+-----------------+-----+--------------------+--------------------+----------+
airline_sentiment|label| text| probability|prediction|
+-----------------+-----+--------------------+--------------------+----------+
 negative| 0.0|@SouthwestAir its...|[0.95973562504797...| 0.0|
 positive| 2.0|@JetBlue Thanks! ...|[0.51157698091259...| 0.0|
 negative| 0.0|@SouthwestAir all...|[0.92071259312434...| 0.0|
 negative| 0.0|.@USAirways we ha...|[0.82290299603207...| 0.0|
 negative| 0.0|@SouthwestAir on ...|[0.92687929582379...| 0.0|
 negative| 0.0|.@united call my ...|[0.74202270064336...| 0.0|
 negative| 0.0|@united Broken en...|[0.93953051011625...| 0.0|
 negative| 0.0|@SouthwestAir Hop...|[0.50235794116664...| 0.0|
 negative| 0.0|@SouthwestAir bee...|[0.96176318840813...| 0.0|
 negative| 0.0|@SouthwestAir Guy...|[0.95780228190740...| 0.0|
 negative| 0.0|@USAirways great ...|[0.48482027433862...| 2.0|
 negative| 0.0|@SouthwestAir tri...|[0.80527555672622...| 0.0|
 negative| 0.0|@united THRILLED ...|[0.96782757971507...| 0.0|
 negative| 0.0|Yes, I filed a re...|[0.98572586284368...| 0.0|
 negative| 0.0|@USAirways @Ameri...|[0.62143964597033...| 0.0|
 neutral| 1.0|@JetBlue what's t...|[0.34242304546388...| 1.0|
 neutral| 1.0|@JetBlue I have a...|[0.53526602855914...| 0.0|
 negative| 0.0|@JetBlue oh hell ...|[0.80772305677667...| 0.0|
 negative| 0.0|@USAirways you ha...|[0.66218849399516...| 0.0|
 negative| 0.0|@USAirways your c...|[0.69463677505303...| 0.0|
+-----------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
from pyspark.mllib.evaluation import MulticlassMetrics
results = transformed.select(['prediction', 'label'])
predictionAndLabels=results.rdd
metrics = MulticlassMetrics(predictionAndLabels)

cm=metrics.confusionMatrix().toArray()
accuracy=(cm[0][0]+cm[1][1])/cm.sum()
precision=(cm[0][0])/(cm[0][0]+cm[1][0])
recall=(cm[0][0])/(cm[0][0]+cm[0][1])
#print("Logistic Regression: accuracy,precision,recall",accuracy,precision,recall)
print(f"Logistic Regression metrics")
print(f"Accuracy ", accuracy)
print(f"Precision ", precision)
print(f"Recall ", recall)

Logistic Regression metrics
Accuracy 0.6420727522306109
Precision 0.8180924287118977
Recall 0.9358830146231721

In [0]:
outputstring = "Logistic Regression metrics\n" + "Accuracy " + str(accuracy) + "\nPrecision " + str(precision) + "\nRecall " + str(recall)
#print(outputstring)

In [0]:
access_key = ""

SECRET_KEY = ""

encoded_secret_key = SECRET_KEY.replace("/", "%2F")

aws_bucket_name = "kadentranbigdata"

mount_name = "ktbigdata"

dbutils.fs.mount("s3a://%s:%s@%s" % (access_key, encoded_secret_key, aws_bucket_name), "/mnt/%s" % mount_name)
display(dbutils.fs.ls("/mnt/%s" % mount_name))

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2849415527587097> in <module> 
 9 mount_name = "ktbigdata" 
 10 
 ---> 11 dbutils . fs . mount ( "s3a://%s:%s@%s" % ( access_key , encoded_secret_key , aws_bucket_name ) , "/mnt/%s" % mount_name ) 
 12 display ( dbutils . fs . ls ( "/mnt/%s" % mount_name ) ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 328 exc . __context__ = None 
 329 exc . __cause__ = None 
 --> 330 raise exc
 331 return f_with_exception_handling
 332 

 ExecutionError : An error occurred while calling o10759.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/ktbigdata; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/ktbigdata
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:739)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/ktbigdata
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:418)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$1(MetadataManager.scala:736)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:518)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:725)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:426)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:90)
	at com.databricks.backend.daemon.data.server.handler.CEMountHandler.receive(MountHandler.scala:145)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:98)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:97)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:299)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:258)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:90)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:117)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:117)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:75)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:395)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptur

In [0]:
display(dbutils.fs.ls("/mnt/%s" % mount_name))

path,name,size
dbfs:/mnt/ktbigdata/Tweets.csv,Tweets.csv,3421431


In [0]:
dbutils.fs.put("dbfs:/mnt/ktbigdata/outputpart2.txt", outputstring)


Wrote 110 bytes.
Out[93]: True